In [1]:
# 📦 Vereiste imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

# 🧾 Stap 1 – Laad de data
data = pd.read_excel("..\TrainData.xlsx")

# 🔍 Eerste inspectie
print("Vorm van de data:", data.shape)
print("Kolommen:", data.columns.tolist())
print("Aantal duplicaten:", data.duplicated().sum())
print("Missende waarden per kolom:")
print(data.isnull().sum())

# 🧼 Stap 2 – Dubbele rijen verwijderen
data = data.drop_duplicates()

# 🎯 Stap 3 – Split in features en labels
X = data.drop(columns="label")
y = data["label"]

# 🔁 Zet y om naar numeriek met behoud van pandas Series
label_encoder = LabelEncoder()
y = pd.Series(label_encoder.fit_transform(y), index=y.index)

# ❓ Hoeveel missende waarden blijven over?
print("Totaal aantal missende waarden:", X.isnull().sum().sum())

# ⚠️ Stap 4 – NaNs imputer (mediaan)
X = X.fillna(X.median(numeric_only=True))




<>:18: SyntaxWarning: invalid escape sequence '\T'
<>:18: SyntaxWarning: invalid escape sequence '\T'
C:\Users\jelle\AppData\Local\Temp\ipykernel_21808\4280346610.py:18: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_excel("..\TrainData.xlsx")
C:\Users\jelle\AppData\Local\Temp\ipykernel_21808\4280346610.py:18: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_excel("..\TrainData.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: '..\\TrainData.xlsx'

In [ ]:
# This script implements two machine learning pipelines using the Naive Bayes classifier. 
# The goal is to preprocess the data, perform feature selection, and optimize hyperparameters 
# using grid search with cross-validation. Two different scaling methods (MinMaxScaler and 
# StandardScaler) are applied to compare their effects on model performance. The results of 
# the grid search, including the best hyperparameters, are saved to Excel files for further analysis.

# Pipeline 1

# Initialize MinMaxScaler to scale features to a range between 0 and 1
scaler_minmax = MinMaxScaler()
X_Scaled = scaler_minmax.fit_transform(X)  # Apply scaling to the feature set X

# Initialize Gaussian Naive Bayes classifier
gaussian = GaussianNB()

# Create a pipeline with feature selection and classifier
pipeline = Pipeline([
    ('feature_selection', SelectKBest(chi2)),  # Select top features based on chi-squared test
    ('classifier', gaussian)  # Use Gaussian Naive Bayes as the classifier
])

# Define grid search parameters for hyperparameter tuning
Grid_params = {
    'feature_selection__k': [80, 100, 120, 130, 140],  # Number of top features to select
    'classifier__var_smoothing': [1, 0.1, 0.01, 0.001]  # Smoothing parameter for Naive Bayes
}

# Perform grid search with 5-fold cross-validation and ROC AUC scoring
grid_search = GridSearchCV(pipeline, param_grid=Grid_params, cv=StratifiedKFold(5), scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_Scaled, y)  # Fit the grid search to the scaled data and labels

# Save the grid search results to a DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Export the results to an Excel file
results_df.to_excel("grid_search_Naive_bayes_results_chi.xlsx", index=False)

# Print the best parameters found during grid search
print("Beste parameters:", grid_search.best_params_)


# Pipeline 2

# Initialize StandardScaler to standardize features by removing the mean and scaling to unit variance
scaler_standard = StandardScaler()
X_Scaled = scaler_standard.fit_transform(X)  # Apply scaling to the feature set X

# Initialize Gaussian Naive Bayes classifier
gaussian = GaussianNB()

# Create a pipeline with feature selection and classifier
pipeline = Pipeline([
    ('feature_selection', SelectKBest()),  # Select top features (default scoring method)
    ('classifier', gaussian)  # Use Gaussian Naive Bayes as the classifier
])

# Define grid search parameters for hyperparameter tuning
Grid_params = {
    'feature_selection__k': [80, 100, 120, 130, 140],  # Number of top features to select
    'classifier__var_smoothing': [1, 0.1, 0.01, 0.001]  # Smoothing parameter for Naive Bayes
}

# Perform grid search with 5-fold cross-validation and ROC AUC scoring
grid_search = GridSearchCV(pipeline, param_grid=Grid_params, cv=StratifiedKFold(5), scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_Scaled, y)  # Fit the grid search to the scaled data and labels

# Save the grid search results to a DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Export the results to an Excel file
results_df.to_excel("grid_search_Naive_bayes_results_standard.xlsx", index=False)

# Print the best parameters found during grid search
print("Beste parameters:", grid_search.best_params_)

In [ ]:
# This section performs a grid search to further optimize the hyperparameters of a Naive Bayes classifier.
# The pipeline includes feature selection using SelectKBest and scaling using StandardScaler.
# The grid search evaluates different numbers of selected features (`k`) and smoothing parameters (`var_smoothing`)
# using 5-fold cross-validation with ROC AUC as the scoring metric.
# The results are saved to an Excel file, and the best parameters are printed.

# Laatste gridsearch
scaler_standard = StandardScaler()  # Standardize features by removing the mean and scaling to unit variance
X_Scaled = scaler_standard.fit_transform(X)  # Apply scaling to the feature set X

gaussian = GaussianNB()  # Initialize Gaussian Naive Bayes classifier

# Create a pipeline with feature selection and classifier
pipeline = Pipeline([
    ('feature_selection', SelectKBest()),  # Select top features (number of features determined by grid search)
    ('classifier', gaussian)  # Use Gaussian Naive Bayes as the classifier
])

# Define grid search parameters for hyperparameter tuning
Grid_params = {
    'feature_selection__k': [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135],  # Number of top features to select
    'classifier__var_smoothing': [0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017]  # Smoothing parameter for Naive Bayes
}

# Perform grid search with 5-fold cross-validation and ROC AUC scoring
grid_search = GridSearchCV(pipeline, param_grid=Grid_params, cv=StratifiedKFold(5), scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_Scaled, y)  # Fit the grid search to the scaled data and labels

# Save the grid search results to a DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Export the results to an Excel file
results_df.to_excel("grid_search_Naive_bayes_results_last.xlsx", index=False)

# Print the best parameters found during grid search
print("Beste parameters:", grid_search.best_params_)

In [ ]:
# This section saves the final trained Naive Bayes model along with the selected features and scaler.
# The final model is trained using the top 134 features selected by SelectKBest and a smoothing parameter of 0.011.
# The trained model, selected features mask, and scaler saved as pickle files for future use.

# Opslaan laatste model
scaler_standard = StandardScaler()  # Standardize features by removing the mean and scaling to unit variance
X_Scaled = scaler_standard.fit_transform(X)  # Apply scaling to the feature set X

# Select the top 134 features using SelectKBest
selector = SelectKBest(k=134)
X_kbest = selector.fit_transform(X_Scaled, y)  # Apply feature selection
selected_features_mask = selector.get_support()  # Get the mask of selected features

# Train the Gaussian Naive Bayes classifier with the specified smoothing parameter
gaussian = GaussianNB(var_smoothing=0.011)
gaussian.fit(X_kbest, y)  # Fit the model to the selected features and labels

# Save the trained model to a pickle file
with open(r'BayesFinal\naive_bayes_model.pkl', 'wb') as file:
    pickle.dump(gaussian, file)

# Save the selected features mask to a pickle file
with open(r'BayesFinal\selected_features_mask.pkl', 'wb') as file:
    pickle.dump(selected_features_mask, file)

# Save the scaler to a pickle file
with open(r'BayesFinal\naive_bayes_minmax_Scaler.pkl', 'wb') as file:
    pickle.dump(scaler_standard, file)

In [ ]:
# testcode

clf = GaussianNB()



scores = cross_validate(clf, X_Scaled, y, cv=StratifiedKFold(10), scoring=["roc_auc", "accuracy"])
print(f"AUC = {scores['test_roc_auc'].mean()} and accuracy = {scores['test_accuracy'].mean()}")